In [ ]:
import pandas as pd
import glob,re,os
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = {
    'airvis': pd.read_csv('restaurant/air_visit_data.csv'),
    'airres': pd.read_csv('restaurant/air_reserve.csv'),
    'airinfo': pd.read_csv('restaurant/air_store_info.csv'),
    'hpgres': pd.read_csv('restaurant/hpg_reserve.csv'),
    'hpginfo': pd.read_csv('restaurant/hpg_store_info.csv'),
    're': pd.read_csv('restaurant/store_id_relation.csv'),
    'sample': pd.read_csv('restaurant/sample_submission.csv'),
    'hol': pd.read_csv('restaurant/date_info.csv')
}

In [ ]:
data['hpgres']=data['hpgres'].merge(data['re'],on=['hpg_store_id'],how='inner')

In [ ]:
data['hpgres'] = data['hpgres'].rename(columns={'visit_datetime': 'visit_date'})
data['airres'] = data['airres'].rename(columns={'visit_datetime': 'visit_date'})

In [ ]:
data['hpgres'].head()

In [ ]:
for df in ['airres','hpgres']:
    data[df]['visit_date'] = pd.to_datetime(data[df]['visit_date']).dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime']).dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_date'] - r['reserve_datetime']).days, axis=1)
    temp1 = data[df].groupby(['air_store_id','visit_date'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'reserve_datetime_diff': 'sum of diff', 'reserve_visitors': 'sum of res vistors'})
    temp2 = data[df].groupby(['air_store_id','visit_date'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'reserve_datetime_diff': 'mean of diff', 'reserve_visitors':'mean of res visitors'})
    data[df]=temp1.merge(temp2,how='inner',on=['air_store_id','visit_date'])

In [ ]:
len(data['hpgres'].air_store_id.unique())

In [ ]:
data['airvis']['visit_date']=pd.to_datetime(data['airvis']['visit_date'])
data['airvis']['dow']=data['airvis']['visit_date'].dt.dayofweek
data['airvis']['year']=data['airvis']['visit_date'].dt.year
data['airvis']['month']=data['airvis']['visit_date'].dt.month
data['airvis']['visit_date']=data['airvis']['visit_date'].dt.date

In [ ]:
data['airvis'].head()

In [ ]:
data['sample']['visit_date'] = data['sample']['id'].map(lambda x: str(x).split('_')[2])
data['sample']['air_store_id'] = data['sample']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['sample']['visit_date'] = pd.to_datetime(data['sample']['visit_date'])
data['sample']['dow'] = data['sample']['visit_date'].dt.dayofweek
data['sample']['year'] = data['sample']['visit_date'].dt.year
data['sample']['month'] = data['sample']['visit_date'].dt.month
data['sample']['visit_date'] = data['sample']['visit_date'].dt.date

In [ ]:
data['sample'].tail()

In [ ]:
unique_stores=data['sample']['air_store_id'].unique()
print('The number of unique stores is:', unique_stores.shape[0])
print('total number of data records in test set is',data['sample'].shape[0])

In [ ]:
stores=pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)],
            axis=0,ignore_index=True).reset_index(drop=True)

In [ ]:
stores.tail()

In [ ]:
data['airvis']['id_dow']=data['airvis'].apply(lambda x: '_'.join([str(x['air_store_id']),str(x['dow'])]),axis=1)
data['airvis']=data['airvis'].set_index('id_dow')
data['sample']['id_dow']=data['sample'].apply(lambda x: '_'.join([str(x['air_store_id']),str(x['dow'])]),axis=1)
data['sample']=data['sample'].set_index('id_dow')

In [ ]:
data['sample'].head()

In [ ]:
temp=data['airvis'].groupby(['air_store_id','dow']).agg({'visitors':[np.min, np.mean, np.median, np.max, np.size]}).reset_index()
temp.head()

In [ ]:
temp.columns = ['air_store_id', 'dow', 'min_visitors', 'mean_visitors', 'median_visitors','max_visitors','count_observations']
stores=stores.merge(temp, on=['air_store_id','dow'],how='left')
stores.head()

In [ ]:
stores = pd.merge(stores, data['airinfo'], how='left', on=['air_store_id'])
stores.head()

In [ ]:
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
stores.head()

In [ ]:
from sklearn import *
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

stores['air_genre_name'].unique()

In [ ]:
data['hol']['visit_date']=pd.to_datetime(data['hol']['calendar_date'])
data['hol']['day_of_week']=lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date']=data['hol']['visit_date'].dt.date
data['hol']=data['hol'].drop('calendar_date',axis=1)
#merge the holiday flags to train and test sets.
train=data['airvis'].merge(data['hol'],on=['visit_date'],how='left')
test=data['sample'].merge(data['hol'],on=['visit_date'],how='left')

In [ ]:
train=train.merge(stores,how='left',on=['air_store_id','dow'])
test=test.merge(stores,how='left',on=['air_store_id','dow'])

In [ ]:
for df in ['airres','hpgres']:
    train = pd.merge(train, data[df], on=['air_store_id','visit_date'], how='left')
    test = pd.merge(test, data[df], on=['air_store_id', 'visit_date'], how='left')

In [ ]:
train = train.fillna(-1)
test = test.fillna(-1)

In [ ]:
test.tail()

In [ ]:
import pickle
with open('train.pickle', 'wb') as f:
    pickle.dump(train, f)
with open('test.pickle', 'wb') as f:
    pickle.dump(test, f)

In [ ]:
features = [f for f in train if f not in ['air_store_id', 'visit_date', 'visitors']]

In [ ]:
from sklearn import svm
from sklearn.svm import SVR
clf = svm.SVR()
clf.fit(train[features], train.visitors.values) 
SVR(C=1.0, cache_size=100, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
pred = clf.predict(test[features])

In [ ]:
import pickle
with open('train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('test.pickle', 'rb') as f:
    test = pickle.load(f)

In [ ]:
# select C and gamma
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVR
features = [f for f in train if f not in ['air_store_id', 'visit_date', 'visitors']]
grid = GridSearchCV(SVR(), param_grid={"C":[0.1, 1, 10], "gamma": [1, 0.1, 0.01]}, cv=4)
grid.fit(train[features], train.visitors.values)
print("The best parameters are %s with a score of %0.2f" %(grid.best_params_, grid.best_score_))

In [ ]:
from sklearn import svm
from sklearn.svm import SVR
clf = svm.SVR()
clf.fit(train[features], train.visitors.values) 
SVR(C=grid.best_params_['C'], cache_size=800, coef0=0.0, degree=3, epsilon=0.1, gamma=grid.best_params_['gamma'],
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
pred = clf.predict(test[features])

with open('predict2.pickle', 'wb') as f:
    pickle.dump(pred, f)

In [ ]:
import pickle
with open('train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('test.pickle', 'rb') as f:
    test = pickle.load(f)
with open('predict1.pickle', 'rb') as r:
    test['visitors'] = pickle.load(r)

In [ ]:
sub = test[['id','visitors']].copy()

In [ ]:
sub.to_csv('sub2.csv', index=False)